In [1]:
import os
import matplotlib.pyplot as plt
from keras.utils import load_img
import tensorflow as tf
train_path = "Vegetable Images\train"
validation_path = "Vegetable Images\validation"
test_path = "Vegetable Images\test"

image_categories = os.listdir('Vegetable Images\train')

def plot_images(image_categories):
  plt.figure(figsize=(12, 12))

  for i, cat in enumerate(image_categories):
    image_path = train_path + '/' + cat
    images_in_folder = os.listdir(image_path)
    first_image_of_folder = images_in_folder[0]
    first_image_path = image_path + '/' + first_image_of_folder
    img = load_img(first_image_path)
    img_arr = tf.keras.utils.img_to_array(img)/255.0
    plt.subplot(4, 4, i+1)
    plt.imshow(img)
    plt.title(cat)
    plt.axis('off')

  # Add an empty subplot for the 16th position
  plt.subplot(4, 4, 16)
  plt.axis('off')

  plt.show()

plot_images(image_categories)

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
import tensorflow as tf
from keras.utils import to_categorical

# ... (previous code)

def create_datasets_from_directory(train_path, validation_path, test_path, batch_size=32):
  train_ds = tf.keras.utils.image_dataset_from_directory(
      train_path,
      label_mode='categorical',
      image_size=(256, 256),
      batch_size=batch_size,
      shuffle=True,
      seed=123
  )

  validation_ds = tf.keras.utils.image_dataset_from_directory(
      validation_path,
      label_mode='categorical',
      image_size=(256, 256),
      batch_size=batch_size,
      shuffle=True,
      seed=123
  )

  test_ds = tf.keras.utils.image_dataset_from_directory(
      test_path,
      label_mode='categorical',
      image_size=(256, 256),
      batch_size=batch_size,
      shuffle=True,
      seed=123
  )

  return train_ds, validation_ds, test_ds

def label_encoding(train_ds, validation_ds, test_ds):
  class_names = train_ds.class_names
  num_classes = len(class_names)

  # Print the mapping of class names to integers
  print("Class Name Mapping:")
  for i, class_name in enumerate(class_names):
    print(f"{i}: {class_name}")

  return class_names, num_classes

# Create the datasets
train_ds, validation_ds, test_ds = create_datasets_from_directory(train_path, validation_path, test_path)
#print(test_ds.class_names)
# Perform label encoding
class_names, num_classes = label_encoding(train_ds, validation_ds, test_ds)

In [ ]:
def resize_images(dataset):
  return dataset.map(lambda x, y: (tf.image.resize(x, (150,150)), y))

# Resize images in train_ds, validation_ds, and test_ds
train_ds = resize_images(train_ds)
validation_ds = resize_images(validation_ds)
test_ds = resize_images(test_ds)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Define the CNN model
def create_model(input_shape=(150, 150, 3), num_classes=15):
    model = Sequential()

    # First Conv Block
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Second Conv Block
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Third Conv Block
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    # Fourth Conv Block
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    # Flatten and Dense Layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create the model
model = create_model()

# Display the model architecture
model.summary()


In [ ]:
import keras
early_stopping=keras.callbacks.EarlyStopping(patience=10)

hist=model.fit(train_ds,
              epochs=100,
               verbose=1,
               validation_data=validation_ds,
               callbacks=early_stopping
              )

In [ ]:
h=hist.history
plt.style.use('ggplot')
plt.figure(figsize=(10,5))
plt.plot(h['loss'],c='red',label='Training Loss')
plt.plot(h['val_loss'],c='red',linestyle='--',label='Validation Loss')
plt.plot(h['accuracy'],c='blue',label='Training Accuracy')
plt.plot(h['val_accuracy'],c='blue',linestyle="--",label='Validation Accuracy')
plt.xlabel("Number of Epochs")
plt.legend(loc='best')
plt.show()


In [ ]:
model.evaluate(test_ds)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_preds():
    plt.figure(figsize=(16,16))  # Create the figure outside the loop
    for x, y in train_ds.take(1):
        pass  # Get a batch of data
    for i in range(32):
        plt.subplot(7, 5, i+1)  # Adjusted to plot a 6x6 grid
        predicted_label = np.argmax(model.predict(np.reshape(x[i], (1, 150, 150, 3))))
        plt.imshow(x[i].numpy().astype("uint8"))  # Convert tensor to image format
        plt.title(f"Predicted: {class_names[predicted_label]}, Actual: {class_names[np.argmax(y[i])]}",fontsize=8, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 2})
        plt.axis('off')  # Turn off axis labels for clarity

    #plt.tight_layout()  # Adjust the spacing between plots
    plt.show()  # Show the entire grid after the loop
plot_preds()

In [ ]:
model.save('classifier.h5')